In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_file = open("../../data/iyer/train.txt")

In [3]:
train_file_contents = train_file.readlines()

In [4]:
len(train_file_contents)

52997

In [5]:
train = []
for line in train_file_contents:
    items = line.split('\t')
    if len(items) == 5:
        train.append(line.split('\t')[2])

In [6]:
len(train)

52795

In [7]:
len(train[0])

31

In [8]:
len(train[1])

23

In [9]:
num_words=100

In [10]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<UNK>')

In [11]:
tokenizer.fit_on_texts(train)

In [12]:
train_seqs = tokenizer.texts_to_sequences(train)

In [13]:
train_tensor = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [14]:
train_tensor[0]

array([6, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [15]:
train[0]

'C# getters, setters declaration'

In [16]:
train_tensor = np.array(train_tensor)

In [17]:
input_dim = train_tensor.shape[1]

In [18]:
from mlp_vae import MLPVariationalAutoEncoder

In [19]:
input_dim

31

In [20]:
train_tensor_oh = tf.one_hot(train_tensor, num_words)

In [21]:
train_tensor_oh.shape

TensorShape([52795, 31, 100])

In [22]:
train_mat = tf.reshape(train_tensor_oh, (train_tensor_oh.shape[0], train_tensor_oh.shape[1] * train_tensor_oh.shape[2]))

In [23]:
train_mat.shape

TensorShape([52795, 3100])

In [27]:
model = MLPVariationalAutoEncoder(train_mat.shape[1], 100, [1000])
model.compile(optimizer='adam', loss='mean_squared_error')

In [28]:
history = model.fit(train_mat, train_mat, batch_size=64, epochs=1, verbose=1)

Train on 52795 samples
52795/52795 [==============================] - 13s 239us/sample - loss: 0.0331


In [30]:
pred = model.predict(np.array([train_mat[0]]))

In [33]:
print(pred[0])

[6.2838197e-04 8.3886290e-01 9.5754862e-04 ... 9.9700689e-04 7.0101023e-04
 1.1396408e-03]


In [40]:
np.set_printoptions(threshold=np.inf)

In [41]:
pred[0]

array([6.2838197e-04, 8.3886290e-01, 9.5754862e-04, 6.9421530e-04,
       5.1021576e-04, 7.3712170e-03, 5.8154941e-02, 5.1209331e-04,
       8.2191825e-04, 9.4658136e-04, 8.4823370e-04, 4.5093894e-03,
       6.4620376e-04, 2.0317435e-03, 8.4272027e-04, 2.9896080e-02,
       6.4679980e-04, 6.8271160e-04, 6.6486001e-04, 8.2489848e-04,
       9.3767047e-04, 7.2312355e-04, 9.0861320e-04, 1.0965168e-03,
       4.3085217e-04, 5.6174397e-04, 3.6911368e-03, 6.2546134e-04,
       1.0722876e-03, 9.6094608e-04, 1.4066696e-03, 9.8201632e-04,
       7.3206425e-04, 1.1591911e-03, 9.2667341e-04, 6.7192316e-04,
       8.7633729e-04, 1.0279417e-03, 1.2551546e-03, 1.8407702e-03,
       1.3276041e-03, 7.6285005e-04, 8.6456537e-04, 2.4667084e-03,
       3.1938851e-03, 9.0456009e-04, 1.2783706e-03, 8.8790059e-04,
       8.9815259e-04, 2.6786625e-03, 6.4980984e-04, 1.1287034e-03,
       6.2137842e-04, 1.0039806e-03, 8.8930130e-04, 1.6216040e-03,
       1.4736950e-03, 6.0376525e-04, 1.0883212e-03, 8.6161494e

In [42]:
np.argmax(pred[0])

1200

In [44]:
pred[0,1200]

0.99999994

In [47]:
pred_res = np.reshape(pred, (1, train_tensor_oh.shape[1], train_tensor_oh.shape[2]))

In [49]:
pred_res[0].shape

(31, 100)

In [50]:
np.argmax(pred_res[0,0])

1

In [54]:
np.argmax(pred_res[0,0])

1

In [53]:
np.argmax(train_tensor_oh[0,0])

6

In [59]:
np.argmax(pred_res[0,4])

0